In [31]:
import mysql.connector
from mysql.connector.constants import ClientFlag

In [32]:
config = {
    'user': 'root',
    'password': 'Hh12H8y2ppBAq5Cj',
    'host': 'localhost',
    'database': 'data'
}

cnx = mysql.connector.connect(**config)
cnx.autocommit = True
cursor = cnx.cursor()

# Query 1

In [23]:
cursor.execute("""
SELECT
  q1.AgeGroup,
  q2.Count * 100.0 / q1.Count AS Percentage
FROM
  (
    SELECT
      CASE WHEN party_age BETWEEN 0
      AND 18 THEN 'Underage' WHEN party_age BETWEEN 19
      AND 21 THEN 'Young I' WHEN party_age BETWEEN 22
      AND 24 THEN 'Young II' WHEN party_age BETWEEN 24
      AND 60 THEN 'Adult' WHEN party_age BETWEEN 61
      AND 64 THEN 'Elder I' ELSE 'Elder II' END AS AgeGroup,
      count(1) AS Count
    FROM
      Parties
    GROUP BY
      AgeGroup
  ) q1
  INNER JOIN (
    SELECT
      CASE WHEN p.party_age BETWEEN 0
      AND 18 THEN 'Underage' WHEN p.party_age BETWEEN 18
      AND 21 THEN 'Young I' WHEN p.party_age BETWEEN 21
      AND 24 THEN 'Young II' WHEN p.party_age BETWEEN 24
      AND 60 THEN 'Adult' WHEN p.party_age BETWEEN 60
      AND 64 THEN 'Elder I' ELSE 'Elder II' END AS AgeGroup,
      count(1) AS Count
    FROM
      Parties p
    WHERE
      p.at_fault = true
      AND p.party_type IN (SELECT id FROM PartyType WHERE description='driver')
    GROUP BY
      AgeGroup
  ) q2 ON q1.AgeGroup = q2.AgeGroup""")

In [35]:
cursor.execute("""SELECT
      CASE WHEN party_age BETWEEN 0
      AND 18 THEN 'Underage' WHEN party_age BETWEEN 19
      AND 21 THEN 'Young I' WHEN party_age BETWEEN 22
      AND 24 THEN 'Young II' WHEN party_age BETWEEN 24
      AND 60 THEN 'Adult' WHEN party_age BETWEEN 61
      AND 64 THEN 'Elder I' ELSE 'Elder II' END AS AgeGroup,
      count(1) AS Count
    FROM
      Parties
    GROUP BY
      AgeGroup""")

KeyboardInterrupt: 

In [24]:
res = cursor.fetchall()

In [25]:
for x in res:
    print(x)

('Young I', Decimal('56.28855'))
('Underage', Decimal('56.44656'))
('Adult', Decimal('40.01389'))
('Elder II', Decimal('43.00967'))
('Young II', Decimal('50.83931'))
('Elder I', Decimal('38.93744'))


# Query 4
## Part 1

In [ ]:
cursor.execute("""
SELECT
  q1.SeatingPosition,
  q2.Count / q1.Count AS SafetyIndex
FROM
  (
    SELECT
      CASE WHEN victim_seating_position BETWEEN 1
      AND 1 THEN 'Driver' WHEN victim_seating_position BETWEEN 2
      AND 6 THEN 'Passenger' WHEN victim_seating_position BETWEEN 7
      AND 7 THEN 'Station Wagon Rear' WHEN victim_seating_position BETWEEN 8
      AND 8 THEN 'Rear Occupant of Truck or Van' WHEN victim_seating_position BETWEEN 9
      AND 9 THEN 'Position Unknown' WHEN victim_seating_position BETWEEN 0
      AND 0 THEN 'Other Occupants' WHEN victim_seating_position BETWEEN 'A'
      AND 'Z' THEN 'Bus Occupants' ELSE 'Not Stated' END AS SeatingPosition,
      count(1) AS Count
    FROM
      Victims
    GROUP BY
      SeatingPosition
  ) q1
  INNER JOIN (
    SELECT
      CASE WHEN victim_seating_position BETWEEN 1
      AND 1 THEN 'Driver' WHEN victim_seating_position BETWEEN 2
      AND 6 THEN 'Passenger' WHEN victim_seating_position BETWEEN 7
      AND 7 THEN 'Station Wagon Rear' WHEN victim_seating_position BETWEEN 8
      AND 8 THEN 'Rear Occupant of Truck or Van' WHEN victim_seating_position BETWEEN 9
      AND 9 THEN 'Position Unknown' WHEN victim_seating_position BETWEEN 0
      AND 0 THEN 'Other Occupants' WHEN victim_seating_position BETWEEN 'A'
      AND 'Z' THEN 'Bus Occupants' ELSE 'Not Stated' END AS SeatingPosition,
      count(1) AS Count
    FROM
      Victims
    WHERE
      victim_degree_of_injury IN (SELECT id FROM VictimDegreeOfInjury WHERE description='no injury')
    GROUP BY
      SeatingPosition
  ) q2 ON q1.SeatingPosition = q2.SeatingPosition""")

In [ ]:
res = cursor.fetchall()

In [ ]:
for x in res:
    print(x)

## Part 2

In [27]:
cursor.execute("""
CREATE VIEW SafetyIndexes AS 
    (SELECT
    q1.SeatingPosition,
    q2.Count / q1.Count AS SafetyIndex
    FROM
    (
        SELECT
        CASE WHEN victim_seating_position BETWEEN 1
        AND 1 THEN 'Driver' WHEN victim_seating_position BETWEEN 2
        AND 6 THEN 'Passenger' WHEN victim_seating_position BETWEEN 7
        AND 7 THEN 'Station Wagon Rear' WHEN victim_seating_position BETWEEN 8
        AND 8 THEN 'Rear Occupant of Truck or Van' WHEN victim_seating_position BETWEEN 9
        AND 9 THEN 'Position Unknown' WHEN victim_seating_position BETWEEN 0
        AND 0 THEN 'Other Occupants' WHEN victim_seating_position BETWEEN 'A'
        AND 'Z' THEN 'Bus Occupants' ELSE 'Not Stated' END AS SeatingPosition,
        count(1) AS Count
        FROM
        Victims
        GROUP BY
        SeatingPosition
    ) q1
    INNER JOIN (
        SELECT
        CASE WHEN victim_seating_position BETWEEN 1
        AND 1 THEN 'Driver' WHEN victim_seating_position BETWEEN 2
        AND 6 THEN 'Passenger' WHEN victim_seating_position BETWEEN 7
        AND 7 THEN 'Station Wagon Rear' WHEN victim_seating_position BETWEEN 8
        AND 8 THEN 'Rear Occupant of Truck or Van' WHEN victim_seating_position BETWEEN 9
        AND 9 THEN 'Position Unknown' WHEN victim_seating_position BETWEEN 0
        AND 0 THEN 'Other Occupants' WHEN victim_seating_position BETWEEN 'A'
        AND 'Z' THEN 'Bus Occupants' ELSE 'Not Stated' END AS SeatingPosition,
        count(1) AS Count
        FROM
        Victims
        WHERE
        victim_degree_of_injury IN (SELECT id FROM VictimDegreeOfInjury WHERE description='no injury')
        GROUP BY
        SeatingPosition
    ) q2 ON q1.SeatingPosition = q2.SeatingPosition)""")

In [28]:
cursor.execute("""
SELECT 
    *
FROM 
    SafetyIndexes
WHERE
    SafetyIndex IN (
          (SELECT MIN(SafetyIndex) FROM SafetyIndexes),
          (SELECT MAX(SafetyIndex) FROM SafetyIndexes)
      )""")

In [29]:
res = cursor.fetchall()

In [30]:
for x in res:
    print(x)

('Station Wagon Rear', Decimal('0.8251'))
('Driver', Decimal('0.0090'))
